## Redo everything on 20% sample, training everything in most basic way possible
* First fine tune some models , then do fusion
* Maybe try fully fine-tuning networks

In [1]:
import os
import sys
import glob
import shutil
import random
import pickle
import numpy as np
from PIL import Image
import time
import copy
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.utils.data
from torchvision.models import resnet18, resnet34, resnet50, resnet101, vgg16
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler

# Set it to use GPU1
torch.cuda.set_device(0)
print(torch.cuda.is_available())
print(torch.cuda.current_device())

True
0


### Make 20% sample dataset:

In [2]:
# def make_sample_dataset(data_loc, new_loc, ttv_folders, frac):
#     """ttv_folders are the names of test, train, valid folders. Assume already in fastai folder format"""   
#     for folder in ttv_folders:
#         classes = [name for name in os.listdir(os.path.join(data_loc, folder)) if os.path.isdir(os.path.join(data_loc, folder, name))]
#         # go through classes, make output directory, copy a sample of image to this
#         for class_name in classes:
#             curr_path = os.path.join(data_loc, folder, class_name)
#             out_path = os.path.join(new_loc, folder, class_name)
#             if not os.path.exists(out_path):
#                 os.makedirs(out_path)
                
#             files = glob.glob(curr_path + '/*.png')
#             sample_files = random.sample(files, int(len(files)*frac))
#             for file_to_copy in sample_files:
#                 shutil.copyfile(file_to_copy, os.path.join(new_loc, folder, class_name, file_to_copy.rsplit('/')[-1]))

# PATH = '/media/rene/Data/camelyon_out/tiles_224_100t'
# new_loc = os.path.join(PATH, 'sample')
# ttv_folders = ['train', 'valid']
# frac = .2
# make_sample_dataset(PATH, new_loc, ttv_folders, frac)

In [3]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    use_gpu = True
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)

                # for nets that have multiple outputs such as inception
                if isinstance(outputs, tuple):
                    loss = sum((criterion(o,labels) for o in outputs))
                else:
                    loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    _, preds = torch.max(outputs.data, 1)
                    loss.backward()
                    optimizer.step()
                else:
                    _, preds = torch.max(outputs.data, 1)

                # statistics
                running_loss += loss.data[0] * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
            # stop those memory leaks
            del loss, outputs 

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best valid Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return best_acc, model

In [3]:
def make_batch_gen(PATH, batch_size, num_workers, valid_name='valid'):
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        valid_name: transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    image_datasets = {x: datasets.ImageFolder(os.path.join(PATH, x),
                                              data_transforms[x])
                      for x in ['train', valid_name]}

    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                                 shuffle=True, num_workers=num_workers)
                  for x in ['train', valid_name]}

    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', valid_name]}
    return dataloaders, dataset_sizes


PATH = '/media/rene/Data/camelyon_out/tiles_224_100t/sample'
num_workers = 4
batch_size=32
sz=224

dataloaders, dataset_sizes = make_batch_gen(PATH, batch_size, num_workers)

In [7]:
model_list = [resnet18, resnet34, resnet50, resnet101, vgg16]
model_name = ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'vgg16']

epochs = 12
save_path = '/media/rene/Data/camelyon_out/tiles_224_100t/sample/models'
results = []

for idx, model_arch in enumerate(model_list):
    model_ft = model_arch(pretrained=True)
    if (model_arch==vgg16):
        num_ftrs = model_ft.classifier[0].in_features
        model_ft.classifier = nn.Linear(num_ftrs, 2)
    else:
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = model_ft.cuda()
    
    criterion = nn.CrossEntropyLoss()
    optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)

    best_acc, model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=epochs)
    results.append((model_name[idx], best_acc))
    torch.save(model_ft.state_dict(), os.path.join(save_path, model_name[idx]))

Epoch 0/11
----------
train Loss: 0.2811 Acc: 0.8881
valid Loss: 0.2075 Acc: 0.9210
Epoch 1/11
----------
train Loss: 0.2397 Acc: 0.9029
valid Loss: 0.3004 Acc: 0.8659
Epoch 2/11
----------
train Loss: 0.2259 Acc: 0.9065
valid Loss: 0.2160 Acc: 0.9120
Epoch 3/11
----------
train Loss: 0.1888 Acc: 0.9196
valid Loss: 0.1785 Acc: 0.9269
Epoch 4/11
----------
train Loss: 0.1778 Acc: 0.9270
valid Loss: 0.1804 Acc: 0.9278
Epoch 5/11
----------
train Loss: 0.1714 Acc: 0.9299
valid Loss: 0.1615 Acc: 0.9341
Epoch 6/11
----------
train Loss: 0.1630 Acc: 0.9343
valid Loss: 0.1701 Acc: 0.9320
Epoch 7/11
----------
train Loss: 0.1578 Acc: 0.9359
valid Loss: 0.1648 Acc: 0.9320
Epoch 8/11
----------
train Loss: 0.1568 Acc: 0.9370
valid Loss: 0.1715 Acc: 0.9311
Epoch 9/11
----------
train Loss: 0.1559 Acc: 0.9382
valid Loss: 0.1689 Acc: 0.9311
Epoch 10/11
----------
train Loss: 0.1537 Acc: 0.9380
valid Loss: 0.1661 Acc: 0.9311
Epoch 11/11
----------
train Loss: 0.1544 Acc: 0.9370
valid Loss: 0.1633 Ac

### Performance

In [14]:
def eval_model(model, dataloader, dataset_size, criterion):
    model.train(False)  # Set model to evaluate mode
    model.eval()

    running_loss = 0.0
    running_corrects = 0

    # Iterate over data.
    for data in dataloader:
        # get the inputs
        inputs, labels = data
        inputs = Variable(inputs.cuda())
        labels = Variable(labels.cuda())

        # forward
        outputs = model(inputs)

        # for nets that have multiple outputs such as inception
        if isinstance(outputs, tuple):
            loss = sum((criterion(o,labels) for o in outputs))
        else:
            loss = criterion(outputs, labels)
        
        _, preds = torch.max(outputs.data, 1)
        running_loss += loss.data[0] * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    
    del loss, outputs 
    
    epoch_loss = running_loss / dataset_size
    epoch_acc = running_corrects / dataset_size
    
    print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

In [15]:
model_list = [resnet18, resnet34, resnet50, resnet101, vgg16]
model_name = ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'vgg16']    

save_path = '/media/rene/Data/camelyon_out/tiles_224_100t/sample/models'
dataloader = dataloaders['valid']
dataset_size = dataset_sizes['valid']

criterion = nn.CrossEntropyLoss()


for idx, model_arch in enumerate(model_list):
    # get the proper model architecture
    model_ft = model_arch(pretrained=True)
    if (model_arch==vgg16):
        num_ftrs = model_ft.classifier[0].in_features
        model_ft.classifier = nn.Linear(num_ftrs, 2)
    else:
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = model_ft.cuda()
    
    # load the saved weights
    model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name[idx])))
    print('model: ', model_name[idx])
    eval_model(model_ft, dataloader, dataset_size, criterion)

model:  resnet18
Loss: 0.1615 Acc: 0.9341
model:  resnet34
Loss: 0.1718 Acc: 0.9302
model:  resnet50
Loss: 0.1748 Acc: 0.9305
model:  resnet101
Loss: 0.1900 Acc: 0.9228
model:  vgg16
Loss: 0.2166 Acc: 0.9189


## Fusion

In [6]:
def train_fusion_model(model, model_list, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
        
    for epoch in tqdm(range(num_epochs)):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda())
                    
                ######### Get model outputs
                features = []
                for model_tmp in model_list:
                    output = model_tmp(inputs)
                    features.append(output)
                cat_features = torch.cat(features, 1)
                    
                ###########
                    
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(cat_features)

                # for nets that have multiple outputs such as inception
                if isinstance(outputs, tuple):
                    loss = sum((criterion(o,labels) for o in outputs))
                else:
                    loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    _, preds = torch.max(outputs.data, 1)
                    loss.backward()
                    optimizer.step()
                else:
                    _, preds = torch.max(outputs.data, 1)

                # statistics
                running_loss += loss.data[0] * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                print('saving model with acc ', epoch_acc)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best valid Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [7]:
class WeightedSum(nn.Module):
    def __init__(self):
        super(WeightedSum, self).__init__()
        self.fc1 = nn.Linear(5*2, 2)

    def forward(self, x):
        out = self.fc1(x)
        return out
    
fusion_model = WeightedSum()
fusion_model = fusion_model.cuda()

In [ ]:
sz = 224

model_list = [resnet18, resnet34, resnet50, resnet101, vgg16]
model_name = ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'vgg16']    
save_path = '/media/rene/Data/camelyon_out/tiles_224_100t/sample/models'

batch_size = 4
num_workers = 4
PATH = '/media/rene/Data/camelyon_out/tiles_224_100t/sample'
dataloaders, dataset_sizes = make_batch_gen(PATH, batch_size, num_workers)


model_list_ft = []    
for idx, model_arch in enumerate(model_list):
    # get the proper model architecture
    model_ft = model_arch(pretrained=True)
    if (model_arch==vgg16):
        num_ftrs = model_ft.classifier[0].in_features
        model_ft.classifier = nn.Linear(num_ftrs, 2)
    else:
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = model_ft.cuda()
    
    # load the saved weights
    model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name[idx])))
    model_list_ft.append(model_ft)
    
num_epochs = 5
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(fusion_model.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)


model = train_fusion_model(fusion_model, model_list_ft, 
                    criterion, optimizer_ft, exp_lr_scheduler, num_epochs=num_epochs)
torch.save(model.state_dict(), os.path.join(save_path, 'weighted_sum_fusion'))


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 0/4
----------


train Loss: 0.2052 Acc: 0.9178


 20%|██        | 1/5 [13:12<52:48, 792.17s/it]

valid Loss: 0.2068 Acc: 0.9189
saving model with acc  0.9188622754491018
Epoch 1/4
----------
train Loss: 0.2011 Acc: 0.9190


...

valid Loss: 0.2084 Acc: 0.9186
Epoch 2/4
----------
train Loss: 0.2002 Acc: 0.9181


 60%|██████    | 3/5 [39:38<26:25, 792.69s/it]

valid Loss: 0.2087 Acc: 0.9192
saving model with acc  0.9191616766467066
Epoch 3/4
----------
train Loss: 0.1982 Acc: 0.9206


 80%|████████  | 4/5 [52:50<13:12, 792.69s/it]

valid Loss: 0.2069 Acc: 0.9213
saving model with acc  0.9212574850299401
Epoch 4/4
----------


## Fusion Perfromance

In [10]:
def eval_fusion_model(model, model_list, dataloader, dataset_size, criterion):
    model.train(False)  # Set model to evaluate mode
    model.eval()

    running_loss = 0.0
    running_corrects = 0

    # Iterate over data.
    for data in dataloader:
        # get the inputs
        inputs, labels = data
        inputs = Variable(inputs.cuda())
        labels = Variable(labels.cuda())

        
        ######### Get model outputs
        features = []
        for model_tmp in model_list:
            output = model_tmp(inputs)
            features.append(output)
        cat_features = torch.cat(features, 1)
        ###########
        
        # forward
        outputs = model(cat_features)
        
        # for nets that have multiple outputs such as inception
        if isinstance(outputs, tuple):
            loss = sum((criterion(o,labels) for o in outputs))
        else:
            loss = criterion(outputs, labels)
        
        _, preds = torch.max(outputs.data, 1)
        running_loss += loss.data[0] * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    
    del loss, outputs 
    
    epoch_loss = running_loss / dataset_size
    epoch_acc = running_corrects / dataset_size
    
    print('Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

In [12]:
model_list = [resnet18, resnet34, resnet50, resnet101, vgg16]
model_name = ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'vgg16']    
save_path = '/media/rene/Data/camelyon_out/tiles_224_100t/sample/models'

model_list_ft = []    
for idx, model_arch in enumerate(model_list):
    # get the proper model architecture
    model_ft = model_arch(pretrained=True)
    if (model_arch==vgg16):
        num_ftrs = model_ft.classifier[0].in_features
        model_ft.classifier = nn.Linear(num_ftrs, 2)
    else:
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = model_ft.cuda()
    
    # load the saved weights
    model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name[idx])))
    model_list_ft.append(model_ft)
    
batch_size = 4
num_workers = 4
PATH = '/media/rene/Data/camelyon_out/tiles_224_100t/sample'
dataloaders, dataset_sizes = make_batch_gen(PATH, batch_size, num_workers)

fusion_model = WeightedSum()
fusion_model.load_state_dict(torch.load(os.path.join(save_path, 'weighted_sum_fusion')))
fusion_model = fusion_model.cuda()

dataloader = dataloaders['valid']
dataset_size = dataset_sizes['valid']
criterion = nn.CrossEntropyLoss()
eval_fusion_model(fusion_model, model_list_ft, dataloader, dataset_size, criterion)

Loss: 0.2894 Acc: 0.8907
